In [3]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer, accuracy_score, classification_report
import numpy as np

from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
dataset_link = [
    "../dataset/v2_seperate/ecg.csv",
    "../dataset/v2_seperate/fnirs.csv",
    "../dataset/v2_seperate/mec.csv",
    "../dataset/v2_seperate/skin.csv",
    "../dataset/v2_seperate/survey.csv",
    "../dataset/v2_seperate/v2_eda.csv",
    "../dataset/v2_seperate/v2_original.csv"
]

for link in dataset_link:
    df = pd.read_csv(link)
    
    y = df['diagnosis_type'].values
    y -= 1

    df.drop(columns=['No', 'diagnosis', 'diagnosis_type'], inplace=True)

    X = df.values
    
    seed_li = [1557, 2356, 88488, 616, 821]
    result = 0

    ####XGBoost######################################
    ##################################################
    ##################################################
    
    for seed in seed_li:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=seed
        )

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('xgb', XGBClassifier(
                objective='multi:softprob',
                use_label_encoder=False,
                eval_metric='mlogloss',
                random_state=42
            ))
        ])

        param_grid = {
            'xgb__n_estimators': [100, 200],
            'xgb__learning_rate': [0.01, 0.1],
            'xgb__max_depth': [3, 5],
            'xgb__subsample': [0.8, 1.0],
            'xgb__colsample_bytree': [0.8, 1.0],
            'xgb__gamma': [0, 1],
            'xgb__reg_lambda': [1, 5],
            'xgb__reg_alpha': [0, 1],
            'xgb__min_child_weight': [1, 5]
        }

        f1_scorer = make_scorer(f1_score, average='weighted')
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=f1_scorer,
            cv=cv,
            n_jobs=-1,
            verbose=1
        )

        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        test_f1 = f1_score(y_test, y_pred, average='weighted')
        result += test_f1
        
        accuracy = accuracy_score(y_test, y_pred)

    result = result / 5
    print(link, "XGBoost", result)
    
    import numpy as np
    from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
    from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer
    from catboost import CatBoostClassifier

    #### CatBoost#####################################
    ##################################################
    ##################################################
    result = 0

    for seed in seed_li:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=seed
        )

        model = CatBoostClassifier(
            loss_function='MultiClass',
            random_seed=seed,
            verbose=0
        )

        param_grid = {
            'iterations': [100, 200],
            'learning_rate': [0.01, 0.1],
            'depth': [3, 5],
            'l2_leaf_reg': [1, 5],
            'random_strength': [1, 2], 
            'bagging_temperature': [0, 1], 
            'border_count': [32, 64], 
            'rsm': [0.8, 1.0],   
        }

        f1_scorer = make_scorer(f1_score, average='weighted')

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring=f1_scorer,
            cv=cv,
            n_jobs=-1,
            verbose=1
        )

        grid_search.fit(X_train, y_train)


        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        test_f1 = f1_score(y_test, y_pred, average='weighted')
        result += test_f1
        
    result = result / 5
    print(link, "CatBoost", result)

Fitting 5 folds for each of 512 candidates, totalling 2560 fits


c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\xgboost\core.py:158: UserWarning: [19:42:53] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Fitting 5 folds for each of 512 candidates, totalling 2560 fits


c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\xgboost\core.py:158: UserWarning: [19:42:59] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Fitting 5 folds for each of 512 candidates, totalling 2560 fits


...

Fitting 5 folds for each of 512 candidates, totalling 2560 fits


c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:54] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Fitting 5 folds for each of 512 candidates, totalling 2560 fits


c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\xgboost\core.py:158: UserWarning: [22:06:49] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


../dataset/v2_seperate/v2_original.csv XGBoost 0.5621489075080406
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
../dataset/v2_seperate/v2_original.csv CatBoost 0.625775753315799
